In [1]:
import pandas as pd
import numpy as np
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from category_encoders.leave_one_out import LeaveOneOutEncoder
import re
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef

warnings.simplefilter("ignore")
%matplotlib inline


## Обработаем тестовую выборку ##

In [3]:
df_test = pd.read_csv('test.csv')

In [3]:
df_test

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [5]:
 df_test.isnull().sum()

id                            0
cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [6]:
columns = []
for col in df_test:
    if df_test[col].isnull().sum() <100 and col != 'id':
        columns.append(col)

In [7]:
columns

['cap-diameter',
 'cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'habitat',
 'season']

In [8]:
df_copy_test_ = pd.DataFrame()

In [9]:
for i in columns:
    if i in df_test.columns and df_test[i].dtypes == 'float64':        
        df_copy_test_[i]=df_test[i].fillna(df_test[i].mean())
    else:
#     elif i in df_test.columns and df_test[i].dtypes == 'object':
        df_copy_test_[i]=df_test[i].fillna(df_test[i].mode()[0])

In [10]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [11]:
df_copy_test_.head()

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.64,x,n,t,w,11.13,17.12,w,t,d,a
1,6.90,o,o,f,y,1.27,10.75,n,f,d,a
2,2.00,b,n,f,n,6.18,3.14,n,f,d,s
3,3.47,x,n,f,n,4.98,8.51,w,t,d,u
4,6.17,x,y,f,y,6.73,13.70,y,t,d,u


In [12]:
df_copy_test_.isnull().sum()

cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
habitat                 0
season                  0
dtype: int64

In [13]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [14]:
for column in columns:
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    for i,value in enumerate(df_copy_test_[column]):
        #print(value)
        #print(value)
           
        value = value.replace(" ","")
        
        if re.match("^\d+?\.\d+?$", value) or re.match("^\d+?$", value):
            #print(f"число: {value}") 
            #print(f"мода: {df_copy_test_['cap-shape'].mode()[0]}")
            df_copy_test_[column].loc[i] = df_copy_test_[column].mode()[0]
        elif len(value) > 1:        
            #print(f"не число а строка: {value}:")
            #print(f"не число а строка,мода: {df_copy_test_['cap-shape'].mode()[0]}")
            #i+=1
            df_copy_test_[column].loc[i] = df_copy_test_[column].mode()[0]

        else:
           #print(f"не число: {value}")
           #i = i.append(value)
            pass

#print(i)
    #print(f"{value}: \n{type(value)}")
    

In [15]:
encoder = OneHotEncoder(return_df=True)
X_test = encoder.fit_transform(df_copy_test_)

In [15]:
X_test.shape

(2077964, 164)

## Обработаем тренировочные данные ##

In [16]:
df_train = pd.read_csv('train.csv')

In [17]:
df_train_1 = df_train.copy()

In [18]:
df_train_1.dtypes

id                        int64
class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [19]:
#df_train_1 = df_train.iloc[:1000000]

In [20]:
df_train_1.isnull().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [21]:
df_train_copy = pd.DataFrame()

In [22]:
for column in df_train_1.columns:
    if column == 'class':
        df_train_copy[column] = df_train_1[column]
    if column in columns and df_train_1[column].dtypes == 'float64':
        df_train_copy[column]=df_train_1[column].fillna(df_train_1[column].mean())
    if column in columns and df_train_1[column].dtypes == 'object':
        df_train_copy[column]=df_train_1[column].fillna(df_train_1[column].mode()[0])

In [23]:
df_train_copy.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [24]:
df_train_copy.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
habitat                 0
season                  0
dtype: int64

In [25]:
df_train_copy.head()

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,e,8.80,f,u,f,w,4.51,15.39,w,f,d,a
1,p,4.51,x,o,f,n,4.79,6.48,o,t,d,w
2,e,6.94,f,b,f,w,6.85,9.93,n,f,l,w
3,e,3.88,f,g,f,g,4.16,6.53,w,f,d,u
4,e,5.85,x,w,f,w,3.37,8.36,w,f,g,a


In [26]:
df_train_copy['season'].unique()

array(['a', 'w', 'u', 's'], dtype=object)

In [27]:
df_train_copy.shape

(3116945, 12)

In [28]:
for column in columns:
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    for i,value in enumerate(df_train_copy[column]):
        #print(value)
        #print(value)

        value = value.replace(" ","")

        if re.match("^\d+?\.\d+?$", value) or re.match("^\d+?$", value):
            #print(f"число: {value}") 
            #print(f"мода: {df_copy_test_['cap-shape'].mode()[0]}")
            df_train_copy[column].loc[i] = df_train_copy[column].mode()[0]
        elif len(value) > 1:
            #print(f"не число а строка: {value}:")
            #print(f"не число а строка,мода: {df_copy_test_['cap-shape'].mode()[0]}")
            #i+=1
            df_train_copy[column].loc[i] = df_train_copy[column].mode()[0]

        else:
            #print(f"не число: {value}")
            #i = i.append(value)
            pass

#print(i)
#print(f"{value}: \n{type(value)}") 

In [29]:
y = df_train_copy["class"]
y = y.map({"p":-1, "e":1})

In [30]:
X = df_train_copy.drop(["class"], axis=1)

In [31]:
X.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [32]:
# encoder = LeaveOneOutEncoder(return_df=True)
# X2 = encoder.fit_transform(X,y)
# X2.dtypes

In [32]:
encoder = OneHotEncoder(return_df=True)
X_train = encoder.fit_transform(X,y)

In [34]:
# yencoder = OneHotEncoder(return_df=True)
# X2 = encoder.fit_transform(X)

In [33]:
X_train.shape

(3116945, 165)

In [34]:
X_test.shape

(2077964, 164)

## Посмотрим разницу в уникальных данных по столбцам в наших датасетах ##

In [35]:
train_unique = pd.DataFrame()

In [36]:
for i,column in enumerate(columns):
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    train_unique = pd.concat([train_unique,pd.DataFrame(df_train_copy[column].unique())], ignore_index=True, axis=1)    
     

In [37]:
train_unique.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
dtype: object

In [37]:
train_unique.sort_values(by=5, ascending=True, inplace=True)

In [38]:
train_unique

,0,1,2,3,4,5,6,7
20,i,m,e,l,c,a,o,NaN
5,c,n,b,f,u,c,m,NaN
14,g,d,c,z,z,d,b,NaN
6,s,e,x,p,p,e,u,NaN
0,f,u,f,w,w,f,d,a
7,d,y,p,o,f,g,w,NaN
2,p,b,d,g,n,h,g,u
18,u,c,k,c,s,i,r,NaN
12,l,i,l,d,b,k,k,NaN
8,e,r,g,b,g,l,y,NaN


## Посмотрим на уникальные данные в тестовой выборке ##

In [39]:
test_unique = pd.DataFrame()

In [40]:
for i,column in enumerate(columns):
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    test_unique = pd.concat([test_unique,pd.DataFrame(df_copy_test_[column].unique())], ignore_index=True, axis=1)

In [41]:
test_unique.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
dtype: object

In [42]:
test_unique.sort_values(by=5, ascending=True, inplace=True)

In [43]:
test_unique

,0,1,2,3,4,5,6,7
3,f,k,k,e,g,a,g,w
19,l,c,h,i,c,b,r,NaN
4,s,b,x,f,o,c,h,NaN
13,w,f,e,d,f,d,o,NaN
14,d,s,r,t,h,e,e,NaN
1,o,o,f,y,n,f,p,s
6,c,e,a,g,u,g,l,NaN
18,r,x,g,c,m,h,y,NaN
7,g,g,p,p,e,i,w,NaN
16,u,i,y,a,a,k,c,NaN


In [44]:
columns

['cap-diameter',
 'cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'habitat',
 'season']

In [45]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

## Заменим уникальные значения в тестовых и тренировочных выборках на медианные по столбцам ##

In [46]:
df_test_copy = df_copy_test_

In [47]:
df_test_copy

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.64,x,n,t,w,11.13,17.12,w,t,d,a
1,6.90,o,o,f,y,1.27,10.75,n,f,d,a
2,2.00,b,n,f,n,6.18,3.14,n,f,d,s
3,3.47,x,n,f,n,4.98,8.51,w,t,d,u
4,6.17,x,y,f,y,6.73,13.70,y,t,d,u
...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,w,f,w,2.67,1.35,e,f,d,u
2077960,3.12,x,w,f,w,2.69,7.38,w,f,g,a
2077961,5.73,x,e,f,w,6.16,9.74,y,t,d,a
2077962,5.03,b,n,f,g,6.00,3.46,g,f,d,a


In [48]:
type(df_test_copy)

pandas.core.frame.DataFrame

In [49]:
type(df_copy_test_)

pandas.core.frame.DataFrame

In [50]:
df_copy_test_['has-ring'] = df_copy_test_['has-ring'].replace('b', df_copy_test_['has-ring'].mode()[0])

In [51]:
df_copy_test_.loc[df_copy_test_['has-ring'] == 'b']

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [52]:
df_copy_test_['does-bruise-or-bleed'] = df_copy_test_['does-bruise-or-bleed'].replace('m', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [53]:
df_copy_test_.loc[df_copy_test_['does-bruise-or-bleed'] == 'm']

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [54]:
df_train_copy.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [55]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('d', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [56]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'd']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [57]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('u', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [58]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'u']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [59]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('z', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [60]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'z']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


## Закодируем наши данные 

**Обновим датасеты с данными**

In [61]:
encoder = OneHotEncoder(return_df=True)        # Тестовый датасет
X_test = encoder.fit_transform(df_copy_test_) 

In [62]:
X_test.shape

(2077964, 162)

In [63]:
y_train = df_train_copy["class"]                  # Таргет тренировочного датасета
y_train = y_train.map({"p":-1, "e":1})

In [64]:
X_train = df_train_copy.drop(["class"], axis=1)   # Тренировочный датасет

In [65]:
X_train.shape

(3116945, 11)

In [66]:
encoder = OneHotEncoder(return_df=True)
X_train_2 = encoder.fit_transform(X_train,y_train)

In [67]:
X_test.shape

(2077964, 162)

In [68]:
X_train_2.shape

(3116945, 162)

In [70]:
X_train.head()

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.80,f,u,f,w,4.51,15.39,w,f,d,a
1,4.51,x,o,f,n,4.79,6.48,o,t,d,w
2,6.94,f,b,f,w,6.85,9.93,n,f,l,w
3,3.88,f,g,f,g,4.16,6.53,w,f,d,u
4,5.85,x,w,f,w,3.37,8.36,w,f,g,a


In [72]:
df_train

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [75]:
df_train['cap-shape'] == X_train['cap-shape']

0          True
1          True
2          True
3          True
4          True
           ... 
3116940    True
3116941    True
3116942    True
3116943    True
3116944    True
Name: cap-shape, Length: 3116945, dtype: bool

In [74]:
X_train['cap-shape']

0          f
1          x
2          f
3          f
4          x
          ..
3116940    f
3116941    s
3116942    x
3116943    p
3116944    x
Name: cap-shape, Length: 3116945, dtype: object

In [96]:
i =0
for i, values in enumerate(df_train['cap-shape'][:10]):
    
    if df_train['cap-shape'].iloc[i] != X_train['cap-shape'].iloc[i]:        
        i+= 1
        print(i)
    else:
        pass

In [100]:
X_train['cap-shape'].head(10)

0    f
1    x
2    f
3    f
4    x
5    x
6    p
7    x
8    f
9    x
Name: cap-shape, dtype: object

In [99]:
df_train['cap-shape'].head(10)

0    f
1    x
2    f
3    f
4    x
5    x
6    p
7    x
8    f
9    x
Name: cap-shape, dtype: object

In [101]:
df_train['cap-shape'].iloc[i] != X_train['cap-shape'].iloc[i]

False

In [102]:
df_train['cap-shape'].iloc[i] == X_train['cap-shape'].iloc[i]

True

In [87]:
df_train['cap-shape'].iloc[i]

'x'

In [88]:
X_train['cap-shape'].iloc[i]

'x'

## Разобьем данные ##

In [103]:
X_train_2_2 = X_train_2[:2000000]

In [104]:
y_train_2_2 = y_train[:2000000]

In [105]:
X_train_test = X_train_2[2000000:]

In [106]:
y_train_test = y_train[2000000:]

In [107]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train_2_2, y_train_2_2, test_size=0.3, random_state=42)

## Классифицируем объекты с помощью Light Gradient Boosted Machine (LightGBM)

In [70]:
 from lightgbm import LGBMClassifier

In [77]:
classifier = LGBMClassifier(n_estimators=500,
                            random_state=42,
                            max_bin=128,
                            colsample_bytree=0.6,
                            reg_lambda = 80,
                            learning_rate= 0.1, # Higher LR => Quicker?
                            device = 'gpu',
                            #num_threads=24,
                            verbose = -1).fit(Xtrain, ytrain)

In [78]:
y_pred_lgbm = classifier.predict(Xtest)

In [79]:
print(round(matthews_corrcoef(ytest, y_pred_lgbm),5))

0.95606


In [80]:
y_pred_lgbm_test = classifier.predict(X_train_test)

In [81]:
print(round(matthews_corrcoef(y_train_test, y_pred_lgbm_test),5))

0.95605


In [82]:
print(round(accuracy_score(y_train_test, y_pred_lgbm_test),5))

0.97821


In [83]:
y_pred_lgbm_test_fin = classifier.predict(X_test)

In [84]:
y_kaggle = pd.read_csv('predict_kaggle.csv')

In [86]:
y_kaggle_predict = np.array(y_kaggle['0'])

In [112]:
print(round(matthews_corrcoef(y_kaggle_predict, y_pred_lgbm_test_fin_pd),5))

0.07889


In [102]:
y_pred_lgbm_test_fin_pd = pd.DataFrame(y_pred_lgbm_test_fin)

In [99]:
y_pred_lgbm_test_fin_pd['0'] = y_pred_lgbm_test_fin_pd.replace('-1', 'p')

In [101]:
y_pred_lgbm_test_fin[y_pred_lgbm_test_fin == -1] ='p'

ValueError: invalid literal for int() with base 10: 'p'

In [107]:
y_pred_lgbm_test_fin_pd=y_pred_lgbm_test_fin_pd.replace(-1, 'p')

In [109]:
y_pred_lgbm_test_fin_pd=y_pred_lgbm_test_fin_pd.replace(1, 'e')

In [110]:
y_pred_lgbm_test_fin_pd

,0
0,p
1,p
2,p
3,e
4,p
...,...
2077959,p
2077960,p
2077961,p
2077962,p


In [111]:
y_pred_lgbm_test_fin_np = np.array(y_pred_lgbm_test_fin_pd)

## Обучим модель дерева решений

In [169]:
dt =  DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_split=3, random_state=18)

In [ ]:
dt.fit(Xtrain, ytrain)

y_train_dt = dt.predict(Xtrain)
y_test_dt = dt.predict(Xtest)

In [161]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_dt, ytrain):.8f}")
print(f"Decision tree accuracy test: {accuracy_score(y_test_dt, ytest):.8f}")

Decision tree accuracy train: 0.79657857
Decision tree accuracy test: 0.79760333


In [162]:
y_train_test_dt = dt.predict(X_train_test)

In [163]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_test_dt, y_train_test ):.8f}")

Decision tree accuracy train: 0.79674290


In [164]:
print(f"Decision tree matthews corrcoef: {matthews_corrcoef(y_train_dt, ytrain):.8f}")

Decision tree matthews corrcoef: 0.60983273


In [165]:
print(f"Decision tree matthews corrcoef: {matthews_corrcoef(y_train_test_dt, y_train_test ):.8f}")

Decision tree matthews corrcoef: 0.61011535


In [166]:
y_test_dt = dt.predict(X_test)

In [153]:
y_pred_kaggle = pd.read_csv('predict_kaggle.csv')

In [154]:
y_pred_kaggle['0'] = y_pred_kaggle['0'].replace('p', -1 )

In [155]:
y_pred_kaggle['0'] = y_pred_kaggle['0'].replace('e', 1 )

In [156]:
y_pred_kaggle_np = np.array(y_pred_kaggle['0'])

In [167]:
print(f"Decision tree matthews corrcoef: {matthews_corrcoef(y_pred_kaggle_np, y_test_dt ):.8f}")

Decision tree matthews corrcoef: 0.29337292


In [168]:
print(f"Decision tree accuracy train: {accuracy_score( y_test_dt, y_pred_kaggle_np  ):.8f}")

Decision tree accuracy train: 0.65028942


## Найдем лучшие параметры для Decision Tree

In [108]:
from sklearn.model_selection import GridSearchCV

In [125]:
gscv_dt = GridSearchCV(DecisionTreeClassifier(),
                       {'criterion' : ['gini', 'entropy'],
                        'max_depth': range(15,40)},
                       )

In [126]:
%%time

gscv_dt.fit(X_train_2,y_train)
print(gscv_dt.best_score_)

0.9735930534545844
CPU times: user 4h 46min 53s, sys: 14min 41s, total: 5h 1min 35s
Wall time: 5h 1min 38s


In [115]:
y_train

0         1
1        -1
2         1
3         1
4         1
         ..
999995   -1
999996    1
999997   -1
999998    1
999999    1
Name: class, Length: 1000000, dtype: int64

In [128]:
gscv_rf.best_params_

{'criterion': 'entropy', 'max_depth': 24}

## Посмотрим на ошибку при лучших параметрам модели Decision Tree

In [162]:
X_test_2_8 = X_train_2[:2800000]

In [163]:
y_test_2_8 = y_train[:2800000]

In [165]:
X_train_test_2 = X_train_2[2800000:]

In [166]:
y_train_test_2 = y_train[2800000:]

In [296]:
X_train_test_full = X_train_2

In [298]:
y_test_full = y_train

In [167]:
Xtrain_2_8, Xtest_2_8, ytrain_2_8, ytest_2_8 = train_test_split(X_test_2_8 , y_test_2_8, test_size=0.3, random_state=42)

In [299]:
Xtrain_2_8, Xtest_2_8, ytrain_2_8, ytest_2_8 = train_test_split(X_train_test_full , y_test_full, test_size=0.3, random_state=42)

In [260]:
dt_gsc =  DecisionTreeClassifier(criterion='entropy', max_depth=24,random_state=18)

In [261]:
dt_gsc.fit(Xtrain_2_8, ytrain_2_8)

y_train_2_8_dt_gsc = dt_gsc.predict(Xtrain_2_8)
y_test_2_8_dt_gsc = dt_gsc.predict(Xtest_2_8)

NameError: name 'Xtrain_2_8' is not defined

In [172]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_2_8_dt_gsc, ytrain_2_8):.8f}")
print(f"Decision tree accuracy test: {accuracy_score(y_test_2_8_dt_gsc, ytest_2_8):.8f}")

Decision tree accuracy train: 0.98353061
Decision tree accuracy test: 0.97998571


In [173]:
y_train_test_2_8_dt_gsc = dt.predict(X_train_test_2)

In [174]:
print(f"Decision tree accuracy train - test: {accuracy_score(y_train_test_2_8_dt_gsc, y_train_test_2):.8f}")

Decision tree accuracy train - test: 0.97020303


## Случайный лес

In [106]:
from sklearn.ensemble import RandomForestClassifier

In [82]:
rf = RandomForestClassifier(n_estimators=100, criterion = 'entropy' , max_depth=24, random_state=18)

In [83]:
rf.fit(X_train_2,y_train)

RandomForestClassifier(criterion='entropy', max_depth=24, random_state=18)

In [95]:
y_train_rf = rf.predict(Xtrain)

In [96]:
print(f"Random Forest accuracy train: {accuracy_score(y_train_rf, ytrain ):.8f}")

Random Forest accuracy train: 0.98161857


In [97]:
y_test_rf = rf.predict(Xtest)

In [98]:
print(f"Random Forest accuracy test: {accuracy_score(y_test_rf, ytest ):.8f}")

Random Forest accuracy test: 0.98151167


In [99]:
y_traain_test_rf = rf.predict(X_train_test)

In [89]:
print(f"Random Forest accuracy train - test: {accuracy_score(y_traain_test_rf, y_train_test ):.8f}")

Random Forest accuracy train - test: 0.97165662


## Starting grid search for Random forest

In [90]:
rf_90 = RandomForestClassifier(n_estimators=90, criterion = 'entropy' , max_depth=24, random_state=18)

In [91]:
rf_90.fit(X_train_2,y_train)

RandomForestClassifier(criterion='entropy', max_depth=24, n_estimators=90,
                       random_state=18)

In [94]:
y_train_rf_90 = rf_90.predict(Xtrain)

In [100]:
print(f"Random Forest accuracy train: {accuracy_score(y_train_rf_90, ytrain ):.8f}")

Random Forest accuracy train: 0.98162643


In [101]:
y_test_rf_90 = rf_90.predict(Xtest)

In [102]:
print(f"Random Forest accuracy test: {accuracy_score(y_test_rf_90, ytest ):.8f}")

Random Forest accuracy test: 0.98155833


In [103]:
y_traain_test_rf_90 = rf_90.predict(X_train_test)

In [104]:
print(f"Random Forest accuracy train - test: {accuracy_score(y_traain_test_rf_90 , y_train_test ):.8f}")

Random Forest accuracy train - test: 0.98166069


In [300]:
rf_110 = RandomForestClassifier(n_estimators=110, criterion = 'entropy' , max_depth=24, random_state=18)

In [ ]:
rf_110.fit(X_train_2,y_train)

In [111]:
y_train_rf_110 = rf_110.predict(Xtrain)

In [112]:
print(f"Random Forest accuracy train: {accuracy_score(y_train_rf_110, ytrain ):.8f}")

Random Forest accuracy train: 0.98163929


In [113]:
y_test_rf_110 = rf_110.predict(Xtest)

In [114]:
print(f"Random Forest accuracy test: {accuracy_score(y_test_rf_110, ytest ):.8f}")

Random Forest accuracy test: 0.98153167


In [115]:
y_traain_test_rf_110 = rf_110.predict(X_train_test)

In [116]:
print(f"Random Forest accuracy train - test: {accuracy_score(y_traain_test_rf_110 , y_train_test ):.8f}")

Random Forest accuracy train - test: 0.98171441


In [117]:
rf_110_40 = RandomForestClassifier(n_estimators=110, criterion = 'entropy' , max_depth=40, random_state=18)

In [118]:
rf_110_40.fit(X_train_2,y_train)

RandomForestClassifier(criterion='entropy', max_depth=40, n_estimators=110,
                       random_state=18)

In [119]:
y_train_rf_110_40 = rf_110_40.predict(Xtrain)

In [120]:
print(f"Random Forest accuracy train: {accuracy_score(y_train_rf_110_40, ytrain ):.8f}")

Random Forest accuracy train: 0.99651643


In [121]:
y_test_rf_110_40 = rf_110_40.predict(Xtest)

In [122]:
print(f"Random Forest accuracy test: {accuracy_score(y_test_rf_110_40, ytest ):.8f}")

Random Forest accuracy test: 0.99650500


In [123]:
y_traain_test_rf_110_40 = rf_110_40.predict(X_train_test)

In [124]:
print(f"Random Forest accuracy train - test: {accuracy_score(y_traain_test_rf_110_40 , y_train_test ):.8f}")

Random Forest accuracy train - test: 0.99651550


In [230]:
from sklearn.metrics import matthews_corrcoef

In [231]:
print(f"Random Forest accuracy train: {matthews_corrcoef(y_train_rf_110_40, ytrain ):.8f}")

Random Forest accuracy train: 0.99297381


In [232]:
print(f"Random Forest accuracy test: {matthews_corrcoef(y_test_rf_110_40, ytest ):.8f}")

Random Forest accuracy test: 0.99294899


In [295]:
 print(f"Random Forest accuracy train - test: {matthews_corrcoef(y_train_test,y_traain_test_rf_110_40):.8f}")

Random Forest accuracy train - test: 0.99297157


## Получим значения таргета на тестовых данных

In [168]:
X_test.shape

(2077964, 162)

In [234]:
y_test_110_40 = rf_110_40.predict(X_test)

In [235]:
y_test_110_40.shape

(2077964,)

In [ ]:
# y = df_train_copy["class"]
# y = y.map({"p":-1, "e":1})

In [236]:
y_test_110_40_1 = pd.DataFrame(y_test_110_40)

In [173]:
y_test_110_40_1.columns

Index([0], dtype='int64')

In [237]:
y_test_110_40_1.columns = ['class']

In [175]:
y_test_110_40_1

,class
0,1
1,-1
2,-1
3,-1
4,-1
...,...
2077959,-1
2077960,-1
2077961,-1
2077962,-1


In [251]:
y_test_110_40_1['class'] = y_test_110_40_1['class'].replace( 1 , 'p')

In [252]:
y_test_110_40_1['class'] = y_test_110_40_1['class'].replace( -1 , 'e')

In [241]:
y_test_110_40_1

,class
0,e
1,p
2,p
3,p
4,p
...,...
2077959,p
2077960,p
2077961,p
2077962,p


In [253]:
y_test_110_40_1.insert(0, 'id', df_test['id'])

ValueError: cannot insert id, already exists

In [243]:
y_test_110_40_1.shape

(2077964, 2)

In [254]:
y_test_110_40_1.to_csv("y_test_110_40_1.csv", index =False)

In [245]:
y_test_110_40_1

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [207]:
y_test_110_40_1.to_csv("y_test_110_40_1.csv", index =False)

In [201]:
y_test_110_40_1 = y_test_110_40_1.rename_axis(index=None, columns=None)

In [202]:
y_test_110_40_1

,id,class
0,0,e
1,1,p
2,2,p
3,3,p
4,4,p
...,...,...
2077959,2077959,p
2077960,2077960,p
2077961,2077961,p
2077962,2077962,p


In [255]:
y_test_110_predict = pd.read_csv('y_test_110_40_1.csv')

In [256]:
y_test_110_predict.shape

(2077964, 2)

In [257]:
y_test_110_predict.columns

Index(['id', 'class'], dtype='object')

In [258]:
y_test_110_predict

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [210]:
sample_submission = pd.read_csv('sample_submission.csv')

In [211]:
sample_submission.shape

(2077964, 2)

In [212]:
sample_submission.columns

Index(['id', 'class'], dtype='object')

In [214]:
sample_submission

,id,class
0,3116945,e
1,3116946,e
2,3116947,e
3,3116948,e
4,3116949,e
...,...,...
2077959,5194904,e
2077960,5194905,e
2077961,5194906,e
2077962,5194907,e


In [224]:
y_test_110_40_1  ['id'] = sample_submission['id']

In [225]:
y_test_110_40_1

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [226]:
y_test_110_40_1.to_csv("y_test_110_40_1.csv", index =False)

In [227]:
y_test_110_40_1

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [228]:
test_values = pd.read_csv('test.csv')

In [229]:
test_values 

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [265]:
y_predict_ = rf.predict(X_test)

In [264]:
rf

RandomForestClassifier(criterion='entropy', max_depth=24, random_state=18)

In [274]:
y_predict_dt= pd.DataFrame(y_predict_ )

In [277]:
y_predict_dt.columns = ['class']

In [279]:
y_predict_dt['class'] = y_predict_dt['class'].replace( -1 , 'p')

In [281]:
y_predict_dt['class'] = y_predict_dt['class'].replace( 1 , 'e')

In [283]:
y_predict_dt.insert(0, 'id', df_test['id'])

In [284]:
y_predict_dt

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,e
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [285]:
y_predict_dt.to_csv("y_predict_dt.csv", index =False)

In [286]:
y_predict_values= pd.read_csv('y_predict_dt.csv')

In [289]:
y_predict_values.shape

(2077964, 2)

In [291]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077964 entries, 0 to 2077963
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   class   object
dtypes: int64(1), object(1)
memory usage: 31.7+ MB


In [293]:
y_predict_values.tail()

,id,class
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p
2077963,5194908,e


In [294]:
from sklearn. metrics import matthews_corrcoef